In [3]:
%reload_ext autoreload
%autoreload 2

from book_functions import *
from signin_config import *

from tqdm import tqdm
import pandas as pd
import re
import requests
import signin_config
import pickle
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from imblearn.over_sampling import SMOTE
import time
from selenium.webdriver.chrome.options import Options

Using TensorFlow backend.


**Psuedo-code process:**

top_200_lists = [ agg site urls ]

full_books_list = [ book dictionaries ]

from top_200 :
	200_book_dic = scrape_intial_info(top_200)
	full_books_list += 200_book_dic


for each_dict in full_books_list:
	each_dict += get_new_info( )

	every 50 books:
		latest = full_books_list[ last50 ]
		clean_and_bag_words( latest )
		save_those_50( latest )
		concat_to_saved( latest )
		rewrite_the_saved(saved_list)
 


# Get top 200 list urls from 1909 - 2019

In [79]:
top_200_lists = get_urls()
len(top_200_lists) # should be 111

111

# Inital scrape of books using top 200 lists - returns title, author, rating, num_ratings and id

In [120]:
#add webdriver and initial scrape function
driver = webdriver.Chrome()
initial_gr_signin(driver)

try:
    full_books_list
except:
    full_books_list = []

for top_200_list in top_200_lists[:5]:
    driver.get(top_200_list)
    dic_of_200_books = goodreads_list_scrape(driver)
    full_books_list += dic_of_200_books

In [119]:
first_books_df = pd.DataFrame(full_books_list)
first_books_df.to_csv('books_first_scrape.csv',index=False)

In [ ]:
# run cell only when restarting whole process - shouldn't be run locally anymore
# # pd.DataFrame([]).to_csv('updating_df.csv') 
# # pd.DataFrame([]).to_json('updating_df.json', orient='records') 

# Secondary scrape for img, description, genre, format, pages and isbn

**Also includes continual saving, with space for automated git pushing**

In [106]:
# run these first lines only if above has not already been run - saves time and energy
# driver = webdriver.Chrome()
# initial_gr_signin(driver)

for each_dict in full_books_list:
    idx = full_books_list.index(each_dict)
    
    each_dict.update(secondary_scrape(each_dict['id'],driver))
    each_dict.update(clean_and_bag_words(each_dict))
    
    if idx % 50 == 0 and idx != 0:
        latest = pd.DataFrame(full_books_list[idx-50:idx])
        save_those_50(latest, 'last_50')
        update_saved(add_to_saved_df(latest, 'updating_df.csv'), 'updating_df')
        push_to_git('last_50', 'updating_df',f'added 50 rows to df at idx {idx}')
        

# saves to full_books_list when finished
pd.DataFrame(full_books_list).to_csv('full_books_list.csv', index=False)
pd.DataFrame(full_books_list).to_json('full_books_list.json', orient='records')

FileNotFoundError: [Errno 2] File b'updating_df' does not exist: b'updating_df'